In [ ]:
# Cell 1 - Install dependencies
!pip install git+https://github.com/openai/whisper.git -q
!pip install gradio -q
!pip install transformers accelerate sentencepiece gTTS -q
!pip install tensorflow -q


  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [ ]:
# Cell 2 - Imports & device
import warnings
warnings.filterwarnings("ignore")

import os
import re
import uuid
import pickle
import torch

import whisper
import gradio as gr
from gtts import gTTS

import tensorflow as tf
from transformers import (
    pipeline,
    DistilBertTokenizerFast,
    TFDistilBertForSequenceClassification,
    MBartForConditionalGeneration,
    MBart50TokenizerFast
)

# Device for whisper and pipelines
DEVICE = "cuda:0" if torch.cuda.is_available() else "cpu"
print(f"Device set to use {DEVICE}")


Device set to use cpu


In [ ]:
# Cell 3 - Whisper, English grammar corrector, and simple translation pipelines
# Whisper STT
whisper_model = whisper.load_model("small", device=DEVICE)

# English grammar corrector (T5 fine-tuned)
en_corrector = pipeline(
    "text2text-generation",
    model="vennify/t5-base-grammar-correction",
    device=0 if torch.cuda.is_available() else -1
)

# Helsinki translation pipelines for quick Hindi<->English translation
hi_to_en = pipeline("translation", model="Helsinki-NLP/opus-mt-hi-en")
en_to_hi = pipeline("translation", model="Helsinki-NLP/opus-mt-en-hi")


Device set to use cpu
Device set to use cpu
Device set to use cpu


In [ ]:
# Cell 4 - mBART (alternative Hindi->English translation, used in pipeline)
mbart_model_name = "facebook/mbart-large-50-many-to-many-mmt"
mbart_tokenizer = MBart50TokenizerFast.from_pretrained(mbart_model_name)
mbart_model = MBartForConditionalGeneration.from_pretrained(mbart_model_name)

def translate_hi_to_en(text: str) -> str:
    mbart_tokenizer.src_lang = "hi_IN"
    inputs = mbart_tokenizer(text, return_tensors="pt")
    generated_tokens = mbart_model.generate(
        **inputs,
        forced_bos_token_id=mbart_tokenizer.lang_code_to_id["en_XX"]
    )
    return mbart_tokenizer.batch_decode(generated_tokens, skip_special_tokens=True)[0]


In [ ]:
# Cell A - Inspect saved model folder
import os

SAVE_DIR = "/content/drive/MyDrive/distilbert-mental-health-model"
print("SAVE_DIR:", SAVE_DIR)
print("Exists:", os.path.exists(SAVE_DIR))
print("\nFiles in SAVE_DIR (recursive):")
for root, dirs, files in os.walk(SAVE_DIR):
    level = root.replace(SAVE_DIR, "").count(os.sep)
    indent = " " * 2 * level
    print(f"{indent}{os.path.basename(root)}/")
    for f in files:
        print(f"{indent}  - {f}")


SAVE_DIR: /content/drive/MyDrive/distilbert-mental-health-model
Exists: True

Files in SAVE_DIR (recursive):
distilbert-mental-health-model/
  distilbert-mental-health-model/
    - config.json
    - label_encoder.pkl
    - special_tokens_map.json
    - tf_model.h5
    - tokenizer.json
    - tokenizer_config.json
    - vocab.txt


In [ ]:
from transformers import DistilBertTokenizerFast, TFDistilBertForSequenceClassification
import pickle, os, tensorflow as tf

SAVE_DIR = "/content/drive/MyDrive/distilbert-mental-health-model/distilbert-mental-health-model"

# If vocab.txt missing, copy from base tokenizer
vocab_path = os.path.join(SAVE_DIR, "vocab.txt")
if not os.path.exists(vocab_path):
    print("Adding missing vocab.txt ...")
    base_tok = DistilBertTokenizerFast.from_pretrained("distilbert-base-uncased")
    base_tok.save_pretrained(SAVE_DIR)
    print("✅ vocab.txt and other tokenizer files added to:", SAVE_DIR)
else:
    print("vocab.txt already exists — skipping creation.")

# Reload tokenizer & model to confirm everything works
sentiment_tokenizer = DistilBertTokenizerFast.from_pretrained(SAVE_DIR)
sentiment_model = TFDistilBertForSequenceClassification.from_pretrained(SAVE_DIR)

with open(os.path.join(SAVE_DIR, "label_encoder.pkl"), "rb") as f:
    sentiment_label_encoder = pickle.load(f)

print("✅ Tokenizer + model + label_encoder loaded successfully!")
print("Classes:", list(sentiment_label_encoder.classes_))

# Quick check
text = "I feel hopeless and empty every day."
inputs = sentiment_tokenizer(text, return_tensors="tf", truncation=True, padding=True, max_length=128)
logits = sentiment_model(inputs).logits
probs = tf.nn.softmax(logits, axis=1)
pred = tf.argmax(probs, axis=1).numpy()[0]
conf = float(tf.reduce_max(probs).numpy())
print(f"Example Prediction: {sentiment_label_encoder.classes_[pred]} ({conf:.2%})")


vocab.txt already exists — skipping creation.


TensorFlow and JAX classes are deprecated and will be removed in Transformers v5. We recommend migrating to PyTorch classes or pinning your version of Transformers.
All model checkpoint layers were used when initializing TFDistilBertForSequenceClassification.

All the layers of TFDistilBertForSequenceClassification were initialized from the model checkpoint at /content/drive/MyDrive/distilbert-mental-health-model/distilbert-mental-health-model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFDistilBertForSequenceClassification for predictions without further training.
TensorFlow and JAX classes are deprecated and will be removed in Transformers v5. We recommend migrating to PyTorch classes or pinning your version of Transformers.


✅ Tokenizer + model + label_encoder loaded successfully!
Classes: [np.str_('Anxiety'), np.str_('Bipolar'), np.str_('Depression'), np.str_('Normal'), np.str_('Personality disorder'), np.str_('Stress'), np.str_('Suicidal')]
Example Prediction: Depression (65.62%)


In [ ]:
# Cell 5 - Load saved DistilBERT sentiment/mental-health model
SAVE_DIR = "/content/drive/MyDrive/distilbert-mental-health-model/distilbert-mental-health-model"

sentiment_model = TFDistilBertForSequenceClassification.from_pretrained(SAVE_DIR)
sentiment_tokenizer = DistilBertTokenizerFast.from_pretrained(SAVE_DIR)

with open(os.path.join(SAVE_DIR, "label_encoder.pkl"), "rb") as f:
    sentiment_label_encoder = pickle.load(f)

print("Sentiment model + tokenizer + label encoder loaded successfully")
print("Available classes:", list(sentiment_label_encoder.classes_))


Some layers from the model checkpoint at /content/drive/MyDrive/distilbert-mental-health-model/distilbert-mental-health-model were not used when initializing TFDistilBertForSequenceClassification: ['dropout_19']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some layers of TFDistilBertForSequenceClassification were not initialized from the model checkpoint at /content/drive/MyDrive/distilbert-mental-health-model/distilbert-mental-health-model and are newly initialized: ['dropout_39']
You should probably TRAIN this model on a down-st

Sentiment model + tokenizer + label encoder loaded successfully
Available classes: [np.str_('Anxiety'), np.str_('Bipolar'), np.str_('Depression'), np.str_('Normal'), np.str_('Personality disorder'), np.str_('Stress'), np.str_('Suicidal')]


In [ ]:
# Cell 6 - Utility functions

def clean_text(text: str) -> str:
    text = text.strip()
    text = re.sub(r"\s+", " ", text)
    return text

def predict_sentiment(text: str):
    inputs = sentiment_tokenizer(text, return_tensors="tf", truncation=True, padding=True, max_length=128)
    logits = sentiment_model(inputs).logits
    probs = tf.nn.softmax(logits, axis=1)
    pred_class = tf.argmax(probs, axis=1).numpy()[0]
    confidence = float(probs[0][pred_class].numpy())
    predicted_status = sentiment_label_encoder.classes_[pred_class]
    return predicted_status, confidence

# optional: contraction normalizer used elsewhere in older notebook (kept for compatibility)
def expand_contractions(text: str) -> str:
    text = text.replace("’", "'").replace("‘", "'")
    text_l = text.lower()
    CONTRACTION_MAP = {
        "i'm": "I am",
        "i've": "I have",
        "i'd": "I would",
        "i'll": "I will",
        "can't": "cannot",
        "won't": "will not",
        "n't": " not",
        "'re": " are",
        "'s": " is",
        "'ve": " have",
        "'d": " would",
        "'ll": " will"
    }
    for contraction, expansion in CONTRACTION_MAP.items():
        text_l = re.sub(r"\b{}\b".format(re.escape(contraction)), expansion, text_l)
    # Keep original capitalization for final text (simple approach)
    return text_l


In [ ]:
# Cell 7 - English/Hindi audio processing pipeline
def english_pipeline(audio_filepath):
    result = whisper_model.transcribe(audio_filepath)
    text = clean_text(result["text"])
    detected_lang = result["language"]

    if detected_lang == "en":
        corrected = en_corrector(text, max_length=512)[0]['generated_text']
        corrected = clean_text(corrected)
        status, conf = predict_sentiment(corrected)

        tts = gTTS(corrected, lang="en")
        filename = f"/tmp/{uuid.uuid4()}.mp3"
        tts.save(filename)

        return f"Detected: English\nCorrected: {corrected}\nSentiment: {status} ({conf:.1%})", filename

    elif detected_lang == "hi":
        # Use mBART for better hi->en as in original pipeline
        en_text = translate_hi_to_en(text)
        en_text = clean_text(en_text)

        status, conf = predict_sentiment(en_text)

        hi_text = en_to_hi(en_text)[0]['translation_text']
        hi_text = clean_text(hi_text)

        tts = gTTS(hi_text, lang="hi")
        filename = f"/tmp/{uuid.uuid4()}.mp3"
        tts.save(filename)

        return (
            f"Detected: Hindi\nHindi: {text}\nEnglish: {en_text}\nSentiment: {status} ({conf:.1%})",
            filename
        )

    else:
        return f"Unsupported language detected: {detected_lang}", None


In [ ]:
# Cell 8 - English text input processing
def english_text_input(text):
    text = clean_text(text)
    corrected = en_corrector(text, max_length=512)[0]['generated_text']
    corrected = clean_text(corrected)

    status, conf = predict_sentiment(corrected)

    tts = gTTS(corrected, lang="en")
    filename = f"/tmp/{uuid.uuid4()}.mp3"
    tts.save(filename)

    return f"Corrected: {corrected}\nSentiment: {status} ({conf:.1%})", filename


In [ ]:
# Cell 9 - Hindi text input processing
def hindi_text_input(text):
    text = clean_text(text)
    en_text = hi_to_en(text)[0]['translation_text']
    en_text = clean_text(en_text)

    status, conf = predict_sentiment(en_text)

    hi_text = en_to_hi(en_text)[0]['translation_text']
    hi_text = clean_text(hi_text)

    tts = gTTS(hi_text, lang="hi")
    filename = f"/tmp/{uuid.uuid4()}.mp3"
    tts.save(filename)

    return (
        f"Hindi Input: {text}\nEnglish: {en_text}\nSentiment: {status} ({conf:.1%})",
        filename
    )


In [ ]:
# Cell 10 - Gradio UI
with gr.Blocks() as demo:
    gr.Markdown("## Speech/Text Grammar + Sentiment (English & Hindi)")

    with gr.Tab("English/Hindi Audio"):
        inp_audio = gr.Audio(sources=["microphone", "upload"], type="filepath")
        out_text = gr.Textbox(label="Corrected / Translated Text + Sentiment")
        out_audio = gr.Audio(label="Speech Output", type="filepath")
        btn_audio = gr.Button("Process")
        btn_audio.click(fn=english_pipeline, inputs=inp_audio, outputs=[out_text, out_audio])

    with gr.Tab("English Text"):
        inp_text_en = gr.Textbox(label="Enter English text")
        out_text_en = gr.Textbox(label="Corrected English + Sentiment")
        out_audio_en = gr.Audio(label="Corrected Speech", type="filepath")
        btn_text_en = gr.Button("Process")
        btn_text_en.click(fn=english_text_input, inputs=inp_text_en, outputs=[out_text_en, out_audio_en])

    with gr.Tab("Hindi Text"):
        inp_text_hi = gr.Textbox(label="Enter Hindi text")
        out_text_hi = gr.Textbox(label="English Translation + Sentiment")
        out_audio_hi = gr.Audio(label="Hindi Speech Output", type="filepath")
        btn_text_hi = gr.Button("Process")
        btn_text_hi.click(fn=hindi_text_input, inputs=inp_text_hi, outputs=[out_text_hi, out_audio_hi])

demo.launch()


It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://f837171f7b7d2a9ddf.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
